In [19]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from time import sleep
import sys
import MeCab
import numpy as np
from PIL import Image
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
import warnings
%matplotlib inline
warnings.simplefilter('ignore')

# 歌詞のスクレイピング
参考: https://qiita.com/nekoumei/items/b1afca7cfb9e54303ab4

In [2]:
def scraping_web_page(url):
    sleep(5)
    html = requests.get(url)
    soup = BeautifulSoup(html.content, 'html.parser')
    return soup

In [14]:
# B'zの歌詞は2ページ
urls = [
    'https://www.uta-net.com/artist/134/',
    'https://www.uta-net.com/artist/134/0/2/'
]
artist_dfs = []
for url in urls:
    #曲一覧ページをスクレイピングする
    soup = scraping_web_page(url)
    #htmlをパースして曲名、各曲URL、アーティスト名、作詞、作曲者名を取得する
    contents = []
    contents.append(soup.find_all(href=re.compile('/song/\d+/$')))
    contents.append(soup.find_all(href=re.compile('/song/\d+/$')))
    contents.append(soup.find_all(class_=re.compile('td2')))
    contents.append(soup.find_all(class_=re.compile('td3')))
    contents.append(soup.find_all(class_=re.compile('td4')))
    infomations = []
    for i, content in enumerate(contents):
        tmp_list = []
        for element in content:
            if i == 0:
                tmp_list.append(element.get('href'))
            else:
                tmp_list.append(element.string)
        infomations.append(tmp_list)
    #DataFrameにする
    artist_df = pd.DataFrame({
        'URL': infomations[0],
        'SongName': infomations[1],
        'Artist': infomations[2],
        'Lyricist': infomations[3],
        'Composer': infomations[4]
    })
    #URLにホストネームを付加
    artist_df['URL'] = artist_df['URL'].apply(lambda x: 'https://www.uta-net.com' + x)
    artist_dfs.append(artist_df)
df_all = pd.concat(artist_dfs)

In [24]:
#各曲のページをスクレイピングする
contents_list = []
for i, url in tqdm(df_all['URL'].iteritems(), total=len(df_all)):
    contents_list.append(scraping_web_page(url))
#歌詞、発売日、商品番号をdataframeに格納する
lyrics = []
sales_dates = []
cd_nums = []
for contents in contents_list:
    lyrics.append(contents.find(id='kashi_area').text)
    sales_dates.append(contents.find(id='view_amazon').text[4:14])
    cd_nums.append(contents.find(id='view_amazon').text[19:28])

In [25]:
df_all['Lyric'] = lyrics
df_all['Sales_Date'] = sales_dates
df_all['CD_Number'] = cd_nums
df_all.head()

,Artist,Composer,Lyricist,SongName,URL,Lyric,Sales_Date,CD_Number
0,B'z,Takahiro Matsumoto,Kohshi Inaba,I WANNA DANCE Wicked Beat Style,https://www.uta-net.com/song/54892/,Crush! Crush! Taxi on a busy neon streetGive m...,：1990-06-2,：BMCR-900
1,B'z,松本孝弘,稲葉浩志,あいかわらずなボクら,https://www.uta-net.com/song/7938/,今まで好きなこともしたし たまに我慢もしてきた「あいつはダメだ」 なんてキミ勝手に決めないで...,：1991-11-2,：BMCR-64B
2,B'z,TAK MATSUMOTO,KOSHI INABA,愛と憎しみのハジマリ,https://www.uta-net.com/song/17979/,誰もが愛する者 守ろうとしてる頃 憎しみがスキをうかがう大破綻の雰囲気が世界中を取り巻いて朝...,：2003-09-1,：BMCV-800
3,B'z,松本孝弘,稲葉浩志,愛のバクダン,https://www.uta-net.com/song/25415/,今日も一日 不完全な世界きみが泣かなくても 誰か泣く人がこしらえた 悲しみのからくりなんとか...,：2005-03-0,：BMCV-400
4,B'z,TAK MATSUMOTO,KOSHI INABA,愛のprisoner,https://www.uta-net.com/song/26049/,"I hate you, hate you こらえきれずに夜の闇に吠える 涙流さないあなたのど...",：2000-12-0,：BMCR-704


In [26]:
df_all.to_csv('lyrics_original.csv')